In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import joblib

def load_raw_titanc():
  # Load the Titanic dataset
  titanic_url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
  return pd.read_csv(titanic_url)

def clean_titanic(titanic_data, fill_missing=True):
  df = titanic_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
  # Fill missing values
  if fill_missing:
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
  else:
    df = df.dropna()
  # Encode categorical features
  df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
  df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
  return df

def load_titanic(cleaned=True, Xy=True):
  '''
  Returns the titanic dataset
  '''
  # Get data
  df = load_raw_titanc()
  # Clean data
  if cleaned:
    # Drop useless features
    df = clean_titanic(df)
  # Split Data
  if Xy:
    return df.drop('Survived', axis=1), df['Survived'] # X, y
  return df

def get_data_loaders(X_train, X_test, y_train, y_test, batch_size=32):
  # Convert to Tensors
  X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
  y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
  X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
  y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

  # Create Datasets
  train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
  test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

  # Create DataLoaders
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, test_loader

def train_loop(train_loader, model, optimizer, criterion, device):
  model.train()
  epoch_train_loss = 0.0
  for batch, labels in train_loader:
    batch, labels = batch.to(device), labels.to(device)
    optimizer.zero_grad()
    # Forward Pass
    outputs = model(batch)
    loss = criterion(outputs, labels)
    # Update Parameters
    loss.backward()
    optimizer.step()
    # Save loss
    epoch_train_loss += loss.item() * batch.size(0)
  return epoch_train_loss / len(train_loader.dataset)

def test_loop(test_loader, model, criterion, device):
  model.eval()
  epoch_test_loss = 0.0
  with torch.no_grad():
    for batch, labels in test_loader:
      batch, labels = batch.to(device), labels.to(device)
      # Calculate predictions
      outputs = model(batch)
      # Save loss
      loss = criterion(outputs, labels)
      epoch_test_loss += loss.item() * batch.size(0)
  return epoch_test_loss / len(test_loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
     

cpu


In [4]:
import wandb
wandb.login() # Login

True

In [5]:
# Define hyperparameters
n_estimators = 30
max_depth = 2

# Create a new run in a project
run = wandb.init(
    project="Simple Scikit-Learn Run",
    notes="commit message for the run",
    config={
        "n_estimators": n_estimators,
        "max_depth": max_depth
    }
)

# Get Data
X, y = load_titanic()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)



/tmp/ipykernel_11070/2799133039.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/tmp/ipykernel_11070/2799133039.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [7]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,0,28.0,1,1,15.2458,0
439,2,0,31.0,0,0,10.5000,2
840,3,0,20.0,0,0,7.9250,2
720,2,1,6.0,0,1,33.0000,2
39,3,1,14.0,1,0,11.2417,0


In [8]:
y_test.head()

709    1
439    0
840    0
720    1
39     1
Name: Survived, dtype: int64

In [12]:
print(X_test.shape)
print(y_test.shape)
X_test?

(179, 7)
(179,)


Type:        DataFrame
String form:
Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
           709       3    0  28.0      1      1  15. <...> 10.5000         2
           10        3    1   4.0      1      1  16.7000         2
           
           [179 rows x 7 columns]
Length:      179
File:        ~/miniconda3/envs/pytorch_cpu_0606/lib/python3.12/site-packages/pandas/core/frame.py
Docstring:  
Two-dimensional, size-mutable, potentially heterogeneous tabular data.

Data structure also contains labeled axes (rows and columns).
Arithmetic operations align on both row and column labels. Can be
thought of as a dict-like container for Series objects. The primary
pandas data structure.

Parameters
----------
data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
    Dict can contain Series, arrays, constants, dataclass or list-like objects. If
    data is a dict, column order follows insertion-order. If a dict contains Series
    which have an index defined, it is al

In [9]:
# Define Random Forest Classifier
clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
clf.fit(X_train, y_train)

# Log metrics
wandb.log({"accuracy": clf.score(X_test, y_test)})

# Finish the run
wandb.finish()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


accuracy,▁
accuracy,0.76536


In [13]:
# Define hyperparameters
num_epochs = 10
learning_rate = 0.01
run_count = 3

# Start a run
run = wandb.init(
    project="Simple PyTorch Run",
    name=f"MyRun{run_count}",
    config={
        "num_epochs": num_epochs,
        "learning_rate": learning_rate
    }
)

# Get Data
X, y = load_titanic()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)
train_loader, test_loader = get_data_loaders(X_train, X_test, y_train, y_test)

# Define Simple PyTorch Model
model = nn.Sequential(
    nn.Linear(7, 32), # 7 input features -> 32 hidden nodes
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  train_loss = train_loop(train_loader, model, optimizer, criterion, device)
  test_loss = test_loop(test_loader, model, criterion, device)

  print(f"[Epoch {epoch+1}/{num_epochs}] TrainLoss: {train_loss}; TestLoss: {test_loss}")
  wandb.log({
    "train_loss": train_loss,
    "test_loss": test_loss
  })

wandb.finish()

/tmp/ipykernel_11070/2799133039.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/tmp/ipykernel_11070/2799133039.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

[Epoch 1/10] TrainLoss: 0.648127836457799; TestLoss: 0.5626600641112088
[Epoch 2/10] TrainLoss: 0.6115628474214104; TestLoss: 0.5837152620267602
[Epoch 3/10] TrainLoss: 0.6230819995483655; TestLoss: 0.5837537396553508
[Epoch 4/10] TrainLoss: 0.5951644028170725; TestLoss: 0.5513184679953079
[Epoch 5/10] TrainLoss: 0.5808188794703966; TestLoss: 0.5310177120416524


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


[Epoch 6/10] TrainLoss: 0.5662129715587316; TestLoss: 0.5027954045620711
[Epoch 7/10] TrainLoss: 0.5370481118057551; TestLoss: 0.4892665602308412
[Epoch 8/10] TrainLoss: 0.5055002412769232; TestLoss: 0.4656464615680652
[Epoch 9/10] TrainLoss: 0.5129898893699217; TestLoss: 0.5087312491579429
[Epoch 10/10] TrainLoss: 0.49310840850465754; TestLoss: 0.4692679610332297


test_loss,▇██▆▅▃▂▁▄▁
train_loss,█▆▇▆▅▄▃▂▂▁
test_loss,0.46927
train_loss,0.49311
